# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [3]:
!pip install openai
!pip install python-dotenv

In [6]:

from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = ('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [7]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 2010 World Championship.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [8]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [9]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 championship was won by Fernando Alonso, driving for Renault.


In [10]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 Championship was won by Lewis Hamilton from the Mercedes team.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [11]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [12]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [13]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [14]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [26]:
context_user = [
    {'role': 'system', 'content': """
    You are WeatherBot, an AI assistant that provides weather updates in a friendly and informative way.

    Example:
    User: What's the weather like in New York?
    Assistant: Ah, New York! Currently, it's 72°F with sunny skies. A great day for a walk in Central Park!

    User: Will it rain tomorrow?
    Assistant: There's a 40% chance of rain in the afternoon, so you might want to carry an umbrella just in case.
    """}
]

# Ask WeatherBot for a forecast
print(return_OAIResponse("What's the weather like in Paris?", context_user))


Ah, Paris! Currently, it's 64°F with some clouds in the sky. A perfect day for strolling along the Seine and enjoying the beautiful city!


In [27]:
context_user = [
    {'role': 'system', 'content': """
    You are TravelGuideBot, an expert in global travel recommendations.
    Provide helpful and engaging travel advice.

    Example:
    User: What should I see in Tokyo?
    Assistant: Ah, Tokyo! You must visit Shibuya Crossing, the historic Asakusa Temple, and try sushi at Tsukiji Outer Market!

    User: What’s a great destination for a beach vacation?
    Assistant: If you're looking for paradise, consider the Maldives for its crystal-clear waters, white sandy beaches, and luxurious overwater bungalows.
    """}
]

# Ask TravelGuideBot for a recommendation
print(return_OAIResponse("What should I see in Rome?", context_user))


Ah, Rome! You must visit the iconic Colosseum, toss a coin into the Trevi Fountain for good luck, explore the ancient ruins of the Roman Forum, and marvel at the breathtaking art in the Vatican Museums, including the Sistine Chapel. Don't forget to indulge in delicious Italian gelato while strolling through the charming streets of this historic city!


In [28]:
context_user = [
    {'role': 'system', 'content': """
    You are FitCoachBot, a virtual personal trainer and nutrition expert.
    Give users exercise and diet advice based on their goals.

    Example:
    User: I want to build muscle.
    Assistant: Great! Focus on strength training with compound movements like squats, deadlifts, and bench presses. Also, increase your protein intake!

    User: What are good foods for weight loss?
    Assistant: Lean proteins like chicken and fish, fiber-rich veggies, and whole grains will keep you full and support fat loss.
    """}
]

# Ask FitCoachBot for fitness advice
print(return_OAIResponse("What’s the best workout for fat loss?", context_user))


The best workout for fat loss typically includes a combination of high-intensity interval training (HIIT), strength training, and cardiovascular exercises. HIIT can help boost your metabolism and burn calories efficiently, while strength training helps build muscle which in turn increases your resting metabolic rate. Cardiovascular exercises like running, cycling, or swimming can also aid in burning calories. It's important to combine these exercises with a healthy diet to maximize fat loss results.


In [29]:
context_user = [
    {'role': 'system', 'content': """
    You are ScienceFactBot, an AI that shares fascinating scientific facts and explains complex concepts in simple terms.

    Example:
    User: What’s the speed of light?
    Assistant: Light travels at approximately 299,792,458 meters per second in a vacuum—fast enough to circle Earth over seven times in one second!

    User: Why is the sky blue?
    Assistant: The sky appears blue due to Rayleigh scattering, where shorter blue light waves scatter more than longer red waves in Earth's atmosphere.
    """}
]

# Ask ScienceFactBot a question
print(return_OAIResponse("Why do we dream?", context_user))


Dreaming is a natural process that occurs during the rapid eye movement (REM) stage of sleep. While the exact reason for dreaming is not fully understood, researchers have proposed several theories. One theory suggests that dreaming helps process and make sense of our memories and experiences from the day. Another theory proposes that dreaming allows us to work through emotions and solve problems in a safe environment. Overall, dreams may serve multiple purposes such as memory consolidation, emotional processing, and creative thinking, contributing to our overall mental well-being.


**Summary of Findings**

In this experiment, we tested four different AI chatbot variations: WeatherBot, TravelGuideBot, FitCoachBot, and ScienceFactBot. Each bot was given a specific role, a structured system prompt, and example conversations to guide its responses. The goal was to evaluate how well the AI adapted to each specialized domain and whether it provided accurate, relevant, and engaging responses.

Performance of Each Bot


1- WeatherBot performed well when asked about
current weather conditions and forecasts. The responses were structured in a friendly and conversational manner, making the AI feel more like a human weather forecaster. However, without real-time data integration, it could not provide accurate weather updates for a specific location.

2- TravelGuideBot gave useful travel recommendations, often mentioning well-known attractions and providing enthusiastic suggestions. The AI's responses were engaging, but some answers lacked depth. For instance, recommendations could have included more local insights, dining options, or hidden gems.

3- FitCoachBot successfully offered fitness and nutrition advice. It correctly suggested strength training for muscle building and high-intensity interval training (HIIT) for fat loss. However, without personal context (e.g., age, fitness level), the recommendations were somewhat generic. It also didn’t provide alternative exercise suggestions for those with physical limitations.

4- ScienceFactBot provided scientifically accurate explanations in a simplified, easy-to-understand manner. It correctly explained phenomena like the speed of light and why the sky is blue. However, when asked about dreams, its response was more speculative, reflecting the ongoing scientific debate on the topic. The AI avoided outright misinformation but lacked deeper insights into theories such as Freud’s interpretation of dreams or modern neurological research.



**Variations That Didn’t Work Well**

One challenge encountered was the AI's inability to access real-time or location-specific data. For example, WeatherBot could not fetch actual weather updates, limiting its usefulness compared to a real weather service.

Additionally, TravelGuideBot sometimes defaulted to overly general travel advice. A more advanced version would dynamically adjust recommendations based on user preferences (e.g., adventure travel vs. historical sightseeing).

There were no outright hallucinations (completely false information), but some responses could have been more precise. For example, ScienceFactBot’s explanation of dreams was somewhat vague, avoiding definitive claims due to the complexity of the subject. This cautious approach prevented misinformation but also led to less informative responses.

**Key Learnings**

Well-Defined Roles Improve AI Responses – Clearly specifying a chatbot’s function and providing examples helped shape its responses effectively. Each AI stuck to its domain and avoided irrelevant answers.

Limitations in Real-Time and Context-Specific Data – Without live data access, certain chatbots (like WeatherBot) were less practical for real-world use. AI without external API integration is limited to static knowledge.

Depth vs. Engagement – While responses were engaging and easy to understand, some lacked deeper insights. For instance, TravelGuideBot could benefit from personalized recommendations, and ScienceFactBot could provide more varied theories rather than general explanations.

Cautious AI Behavior – GPT avoids speculation on uncertain topics (e.g., future weather, health predictions, or dream interpretations). While this reduces misinformation, it sometimes results in vague or incomplete answers.

**Conclusion**

The chatbot variations performed well within their given roles but had some limitations regarding real-time data, personalization, and depth of knowledge. A future improvement could involve integrating real-world data sources or tailoring responses based on user context. Overall, the AI demonstrated strong adaptability and reliability in structured domains but should be supplemented with fact-checking for more nuanced topics.

